This notebook contains the code for preprocessing data from the series of earthquakes that occurred near the city of L'Aquila in Italy in 2008. The data contains info about the location and charateristics of buildings in the area, as well as the damage reported. This is quite messy data, so we will need to do a lot of cleaning and processing to get it into a usable form. 

First of all, we need to import the necessary libraries. We will be using pandas for data manipulation and numpy for numerical operations, among others. 

In [64]:
import pandas as pd
from tqdm import tqdm
from nltk import flatten
import numpy as np
import matplotlib.pyplot as plt
import copy
import pickle

Let's start by importing the data and taking a look 

In [65]:
dataframe =  pd.read_excel('aquila.xls')
dataframe.head()

,identificativo,numerorichiesta,codiceaggregatostrutturale,codiceedificiostrutturale,comune,codicecomune,localita,codicelocalita,identificativoposizioneedificio,coordinate_lat,...,sez3_identificativocopertura,sez4_danno_strutturale_strutture_verticali,sez4_danno_strutturale_solai,sez4_danno_strutturale_scale,sez4_danno_strutturale_copertura,sez4_danno_strutturale_tamponature_tramezzi,sez5_danno_elementi_non_strutturali,sez6_pericolo_esterno,sez7_dissesti_geologico_tecnici,sez7_morfologia_versante
0,10,1.106610e+16,0010099999,NaN,TIONE DEGLI ABRUZZI,66100.0,TIONE DEGLI ABRUZZI,66100.0,Angolo,42203395.0,...,Non spingente pesante,Danno Nullo,Danno Nullo,Danno Nullo,Danno Nullo,Danno Nullo,NaN,Assente,Assenti,Pendio leggero
1,100,1.106800e+16,6800599999,NaN,BUSSI SUL TIRINO,68005.0,BUSSI SUL TIRINO,680051002.0,NaN,422135849.0,...,Non spingente leggera,Danno D1 Leggero:<1/3,Danno D1 Leggero:<1/3,Danno Nullo,Danno Nullo,Danno D1 Leggero:<1/3,NaN,Assente,Assenti,Pendio leggero
2,1000,1.106604e+16,6604302213,001,FONTECCHIO,66043.0,FONTECCHIO,66043.0,Angolo,422299396.0,...,NaN,Danno D4-D5 Gravissimo:<1/3,Danno D4-D5 Gravissimo:<1/3,Danno Nullo,Danno Nullo,Danno Nullo,Presente,Presente,Assenti,Pendio forte
3,10000,1.106609e+16,6608700976,003,SAN DEMETRIO NE' VESTINI,66087.0,SAN DEMETRIO NE' VESTINI,66087.0,Interno,422888836.0,...,Spingente pesante,Danno D1 Leggero:<1/3,Danno D1 Leggero:<1/3,Danno Nullo,Danno Nullo,Danno Nullo,Presente,Assente,Assenti,Pianura
4,10001,1.106609e+16,0008700271,001,SAN DEMETRIO NE' VESTINI,66087.0,SAN DEMETRIO NE' VESTINI,660871001.0,Interno,422892088.0,...,Non spingente leggera,"Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio...","Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:...",Danno Nullo,Danno D1 Leggero:<1/3,Danno Nullo,Presente,Assente,Assenti,Pendio leggero


As we can see, this is quite messy. There are a lot of missing values, and the data is not in a usable form. We will need to do some cleaning and processing to get it into a usable form. Moreover, many columns are not needed for our analysis, so we will drop them. Let's start by converting the coordinates to a usable form.

In [66]:
coordinates_columns = ['coordinate_lat', 'coordinate_lon']

def coord_to_float(coord):
    coord = str(coord)
    if coord != 'nan':
        #split after dot and take first element
        coord = coord.split('.')[0]
        # add a dot after second number
        coord = coord[:2] + '.' + coord[2:]
        return float(coord)
    else:
        return float(coord)

PROPERTIES = []
for column in coordinates_columns:
    dataframe[column] = dataframe[column].apply(coord_to_float)
    #drop rows with 0.0 coordinates
    dataframe = dataframe.dropna(subset=[column])
    
    #normalize coordinates
    mean_coord = dataframe[column].mean()
    std_coord = dataframe[column].std()
    data_properties = {'mean': mean_coord, 'std': std_coord, 'max': dataframe[column].max(), 'min': dataframe[column].min()}
    PROPERTIES.append(data_properties)
    dataframe[column] = (dataframe[column] - mean_coord) / std_coord
    
    print(dataframe[column].head())

0   -0.867421
1   -0.789493
2   -0.664419
3   -0.213638
4   -0.211151
Name: coordinate_lat, dtype: float64
0    0.596896
1    1.602386
2    0.440534
3    0.191047
4    0.169190
Name: coordinate_lon, dtype: float64


In [67]:
PROPERTIES

[{'mean': 42.316818950803125,
  'std': 0.13075990982450972,
  'max': 42.7362168,
  'min': 41.7158661414168},
 {'mean': 13.521897196462286,
  'std': 0.1894015325303454,
  'max': 14.2035408,
  'min': 13.0}]

Now that we have the coordinates in a usable form, we drop the columns that we don't need and reorganize the data into xs and ys.

In [68]:
x_columns = ['coordinate_lat', 
    'coordinate_lon',
    'identificativoposizioneedificio',
    # 'sez3_regolarita2',
    # 'sez3_rinforzata',
    'sez3_struttura_orizzontale_1',
    'sez2_altezzamediapiano',
    'sez2_pianiinterrati',
    # 'sez3_mista',
    'sez3_struttura_verticale_1',
    'sez2_numeropiani',
    # 'sez3_catene_o_cordoli_1',
    'sez2_superficiepiano',
    # 'sez3_regolarita1',
    'sez3_pilastriisolati',
    # 'sez3_struttura_verticale_2',
    'sez2_costruzioneristrutturazione1',
    # 'sez3_catene_o_cordoli_2',
    # 'sez3_struttura_orizzontale_2',
    # 'sez2_costruzioneristrutturazione2',
    'sez7_morfologia_versante'
    ]
# y_columns = ['sez4_danno_strutturale_strutture_verticali',
#     'sez4_danno_strutturale_scale',
#     'sez4_danno_strutturale_tamponature_tramezzi',
#     'sez4_danno_strutturale_copertura',
#     'sez4_danno_strutturale_solai'
#     ]
y_columns = ['sez4_danno_strutturale_strutture_verticali']

Y_SIZE = len(y_columns)
# reorganize dataframe ##########################################################
dataframe_x = dataframe[x_columns]
dataframe_y = dataframe[y_columns]
dataframe = pd.concat([dataframe_x, dataframe_y], axis=1)
# print(len(dataframe.columns))
dataframe.head(20)

,coordinate_lat,coordinate_lon,identificativoposizioneedificio,sez3_struttura_orizzontale_1,sez2_altezzamediapiano,sez2_pianiinterrati,sez3_struttura_verticale_1,sez2_numeropiani,sez2_superficiepiano,sez3_pilastriisolati,sez2_costruzioneristrutturazione1,sez7_morfologia_versante,sez4_danno_strutturale_strutture_verticali
0,-0.867421,0.596896,Angolo,Volte senza catene,2.50-3.50,0,Muratura buona qualità,2,50-70,no,<1919,Pendio leggero,Danno Nullo
1,-0.789493,1.602386,NaN,Volte senza catene,2.50-3.50,Non compilato,Muratura cattiva qualità,4,70-100,no,<1919,Pendio leggero,Danno D1 Leggero:<1/3
2,-0.664419,0.440534,Angolo,Volte senza catene,2.50-3.50,0,Muratura cattiva qualità,3,50-70,no,<1919,Pendio forte,Danno D4-D5 Gravissimo:<1/3
3,-0.213638,0.191047,Interno,Travi con soletta rigida,2.50-3.50,Non compilato,Non identificata,2,<50,no,1992-2001,Pianura,Danno D1 Leggero:<1/3
4,-0.211151,0.169190,Interno,Volte senza catene,2.50-3.50,0,Muratura cattiva qualità,2,130-170,no,<1919,Pendio leggero,"Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio..."
5,-1.416054,1.089482,Angolo,Travi con soletta semirigida,2.50-3.50,1,Struttura Mista telaio-muratura di buona qualità,3,100-130,no,<1919,Pendio leggero,Danno Nullo
6,1.282923,1.002437,Interno,Volte senza catene,2.50-3.50,0,Muratura cattiva qualità,3,50-70,no,<1919,Pendio leggero,Danno Nullo
7,-1.414760,1.089314,Interno,Travi con soletta deformabile,2.50-3.50,0,Muratura cattiva qualità,3,50-70,no,1962-1971,Pianura,Danno Nullo
8,-1.659083,1.520466,Angolo,Volte senza catene,2.50-3.50,0,Muratura cattiva qualità,3,<50,no,<1919,Pianura,Danno D1 Leggero:<1/3
9,-0.208356,0.171897,Interno,Volte senza catene,2.50-3.50,0,Muratura cattiva qualità,3,<50,no,<1919,Pendio leggero,Danno Nullo


In this form, the data is much easier to work with. The last Y_SIZE columns contain the damage values for each building. In this case we choose to study only one type of damage (damage to staircases structures), so we will drop the other columns.

We now count the nans in each column: 

In [69]:
dataframe = dataframe.dropna()
for column in dataframe.columns:
    print(column, dataframe[column].isnull().sum())

coordinate_lat 0
coordinate_lon 0
identificativoposizioneedificio 0
sez3_struttura_orizzontale_1 0
sez2_altezzamediapiano 0
sez2_pianiinterrati 0
sez3_struttura_verticale_1 0
sez2_numeropiani 0
sez2_superficiepiano 0
sez3_pilastriisolati 0
sez2_costruzioneristrutturazione1 0
sez7_morfologia_versante 0
sez4_danno_strutturale_strutture_verticali 0


As almost all of the data is categorical, we save all the unique values for each column in a dictionary.

In [70]:
condensed_categories = {
    'Danno Nullo': 'Danno Nullo',
    
    'Danno D1 Leggero:<1/3': 'Danno Leggero',
    'Danno D1 Leggero:1/3-2/3': 'Danno Leggero',
    'Danno D1 Leggero:>2/3': 'Danno Leggero',
    
    'Danno D2-D3 Medio-Grave:<1/3': 'Danno Medio',
    'Danno D2-D3 Medio-Grave:1/3-2/3': 'Danno Medio',
    'Danno D2-D3 Medio-Grave:>2/3': 'Danno Medio',
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 'Danno Medio',
    'Danno D2-D3 Medio-Grave:1/3-2/3, Danno D1 Leggero:<1/3': 'Danno Medio',
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:>2/3': 'Danno Medio',
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:1/3-2/3': 'Danno Medio',
    
    'Danno D4-D5 Gravissimo:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:1/3-2/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:>2/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:1/3-2/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:>2/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:1/3-2/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:1/3-2/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:>2/3, Danno D2-D3 Medio-Grave:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:>2/3, Danno D1 Leggero:<1/3': 'Danno Grave',
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D1 Leggero:1/3-2/3': 'Danno Grave'
    }

condensed_categories = {
    'Danno Nullo': 0,
    'nan': 0,
    'Danno D1 Leggero:<1/3': 1,
    'Danno D1 Leggero:1/3-2/3': 2,
    'Danno D2-D3 Medio-Grave:<1/3': 3,
    'Danno D1 Leggero:>2/3': 3,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 4,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:1/3-2/3': 5,
    'Danno D2-D3 Medio-Grave:1/3-2/3': 6,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:>2/3': 6,
    'Danno D2-D3 Medio-Grave:1/3-2/3, Danno D1 Leggero:<1/3': 7,
    'Danno D4-D5 Gravissimo:<1/3': 8,
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:<1/3': 9,
    'Danno D2-D3 Medio-Grave:>2/3': 9,
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:1/3-2/3': 10,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3': 11,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 12,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:1/3-2/3': 13,
    'Danno D4-D5 Gravissimo:1/3-2/3': 14,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:>2/3': 15,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D1 Leggero:1/3-2/3': 16,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:<1/3': 17,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:1/3-2/3': 18,
    'Danno D4-D5 Gravissimo:>2/3, Danno D1 Leggero:<1/3': 19,
    'Danno D4-D5 Gravissimo:>2/3, Danno D2-D3 Medio-Grave:<1/3': 19,
    'Danno D4-D5 Gravissimo:>2/3': 19
    }

condensed_categories = {
    'Danno Nullo': 0,
    'nan': 0,
    'Danno D1 Leggero:<1/3': 1,
    'Danno D1 Leggero:1/3-2/3': 2,
    'Danno D2-D3 Medio-Grave:<1/3': 3,
    'Danno D1 Leggero:>2/3': 3,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 4,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:1/3-2/3': 4,
    'Danno D2-D3 Medio-Grave:1/3-2/3': 5,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:>2/3': 5,
    'Danno D2-D3 Medio-Grave:1/3-2/3, Danno D1 Leggero:<1/3': 5,
    'Danno D4-D5 Gravissimo:<1/3': 6,
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:<1/3': 7,
    'Danno D2-D3 Medio-Grave:>2/3': 7,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:>2/3': 7,
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:1/3-2/3': 8,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3': 8,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 8,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:1/3-2/3': 8,
    'Danno D4-D5 Gravissimo:1/3-2/3': 9,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D1 Leggero:1/3-2/3': 9,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:<1/3': 9,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:1/3-2/3': 9,
    'Danno D4-D5 Gravissimo:>2/3, Danno D1 Leggero:<1/3': 10,
    'Danno D4-D5 Gravissimo:>2/3, Danno D2-D3 Medio-Grave:<1/3': 10,
    'Danno D4-D5 Gravissimo:>2/3': 10}


# map categories ###############################################################
dataframe['sez4_danno_strutturale_strutture_verticali'] = dataframe['sez4_danno_strutturale_strutture_verticali'].map(condensed_categories)


In [71]:
dataframe['sez4_danno_strutturale_strutture_verticali'].value_counts()

0     18746
1     11461
3      3718
5      3307
10     2636
9      2339
7      2029
6      1479
2      1364
4       696
8       672
Name: sez4_danno_strutturale_strutture_verticali, dtype: int64

In [72]:


unique_values_dict = {}
for column in dataframe.columns:
    if column in coordinates_columns:
        continue
    column_uniques = dataframe[column].unique() if type(dataframe[column]) == pd.Series else dataframe[column].iloc[:,0].unique()
    unique_values_dict[column] = column_uniques
    
#  sort unique values
unique_values_dict['sez4_danno_strutturale_strutture_verticali'] = sorted(unique_values_dict['sez4_danno_strutturale_strutture_verticali'])

# %% display unique values for each column and save as pkl
print(unique_values_dict)
with open('unique_values_dict_scalar_scores.pkl', 'wb') as f:
    pickle.dump(unique_values_dict, f)

{'identificativoposizioneedificio': array(['Angolo', 'Interno', 'Isolato', 'Estremità'], dtype=object), 'sez3_struttura_orizzontale_1': array(['Volte senza catene', 'Travi con soletta rigida',
       'Travi con soletta semirigida', 'Travi con soletta deformabile',
       'Volte con catene', 'Non identificata'], dtype=object), 'sez2_altezzamediapiano': array(['2.50-3.50', 'Non compilato', '3.50-5.0', '<2.50', '>5.0'],
      dtype=object), 'sez2_pianiinterrati': array([0, 'Non compilato', 1, 2, '>2'], dtype=object), 'sez3_struttura_verticale_1': array(['Muratura buona qualità', 'Muratura cattiva qualità',
       'Non identificata',
       'Struttura Mista telaio-muratura di buona qualità',
       'Struttura Mista telaio-muratura di cattiva qualità'], dtype=object), 'sez2_numeropiani': array([2, 3, 1, 5, 4, 6, 'Non compilato', 7, 8, 11, 10, 9], dtype=object), 'sez2_superficiepiano': array(['50-70', '<50', '130-170', '100-130', 'Non compilato', '70-100',
       '170-230', '400-500', '230-3

In [73]:
dataframe[y_columns[0]].unique() 

array([ 0,  6,  1,  8,  3,  9,  5,  7,  2,  4, 10], dtype=int64)

In [74]:
#Count types of damage
dataframe[y_columns[0]].value_counts()


0     18746
1     11461
3      3718
5      3307
10     2636
9      2339
7      2029
6      1479
2      1364
4       696
8       672
Name: sez4_danno_strutturale_strutture_verticali, dtype: int64

As we said, this data is categorical, so we need to convert it to numerical values. We will use the dictionary we created earlier to do this.

In [75]:
# %% Various functions  to transform data into onehot encoding ##############################################################
def find_unique_positions(column, x, unique_values_dict):
    return  list(unique_values_dict[column]).index(x)
def int_to_onehot(column, x, unique_values_dict):
    onehot = [0]*(len(unique_values_dict[column]))
    onehot[x] = 1
    return onehot

def turn_to_numeric(dataframe, columns_exeptions = [], unique_values_dict = {}):
    for column in dataframe.columns:
        if column in columns_exeptions:
            continue
        print(column)
        dataframe[column] = dataframe[column].apply(lambda x: find_unique_positions(column=column, x=x, unique_values_dict=unique_values_dict))
    return dataframe
def numeric_to_onehot(dataframe, columns_exeptions = [], unique_values_dict = {}):
    for column in dataframe.columns:
        if column in columns_exeptions:
            continue
        dataframe[column] = dataframe[column].apply(lambda x: int_to_onehot(column=column, x=x, unique_values_dict=unique_values_dict))
    return dataframe
def onehot_to_signature(dataframe, Y_SIZE = Y_SIZE):
    signature_df = {'x':[], 'y':[]}
    for i in tqdm(range(len(dataframe))):
        signature_x = [onehot for onehot in dataframe.iloc[i,:-Y_SIZE]]
        signature_y = [onehot for onehot in dataframe.iloc[i,-Y_SIZE:]]
        signature_df['x'].append(flatten(signature_x))
        signature_df['y'].append(flatten(signature_y))
    return pd.DataFrame(signature_df)

In [76]:
dataframe = turn_to_numeric(dataframe, columns_exeptions=coordinates_columns, unique_values_dict = unique_values_dict)
dataframe = numeric_to_onehot(dataframe, columns_exeptions=coordinates_columns, unique_values_dict = unique_values_dict)
dataframe = onehot_to_signature(dataframe)

identificativoposizioneedificio
sez3_struttura_orizzontale_1
sez2_altezzamediapiano
sez2_pianiinterrati
sez3_struttura_verticale_1
sez2_numeropiani
sez2_superficiepiano
sez3_pilastriisolati
sez2_costruzioneristrutturazione1
sez7_morfologia_versante
sez4_danno_strutturale_strutture_verticali


100%|██████████| 48447/48447 [00:34<00:00, 1419.86it/s]


We have now converted the data to numerical values. Let's take a look at it again.

In [77]:
dataframe.head()

,x,y
0,"[-0.8674214516922528, 0.5968959280707078, 1, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[-0.664418864464821, 0.4405339408980041, 1, 0,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,"[-0.21363849853229364, 0.19104651928789274, 0,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[-0.21115149773491346, 0.1691897795630536, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,"[-1.4160536746440862, 1.0894816994400685, 1, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [78]:
# save  dataframe
dataframe.to_pickle('signature_scalar_condensed_dataframe_strutture_verticali.pkl')
dataframe.to_csv('signature_scalar_condensed_dataframe_strutture_verticali.csv', index=False)
dataframe = pd.read_csv('signature_scalar_condensed_dataframe_strutture_verticali.csv')
dataframe.head()

,x,y
0,"[-0.8674214516922528, 0.5968959280707078, 1, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[-0.664418864464821, 0.4405339408980041, 1, 0,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,"[-0.21363849853229364, 0.19104651928789274, 0,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[-0.21115149773491346, 0.1691897795630536, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,"[-1.4160536746440862, 1.0894816994400685, 1, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


This representation is much easier to work with, but the xs are very sparse. We will need to deal with this later.

(DEPRECATED)
In evaluation phase it will be useful to give a numerical score to each damage level. We do this by creating a dictionary that maps each damage level to a score. This is done 'by hand' following a simple rule. 

In [48]:
damage_scores_dict = {'Danno Nullo':0,
                    'Danno D1 Leggero:<1/3':1,
                    'Danno D2-D3 Medio-Grave:<1/3':2,
                    'Danno D4-D5 Gravissimo:<1/3':4,
                    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:1/3-2/3':8,
                    'Danno D4-D5 Gravissimo:1/3-2/3':8,
                    'Danno D2-D3 Medio-Grave:1/3-2/3':4,
                    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:<1/3':10,
                    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:<1/3':5,
                    'Danno D1 Leggero:1/3-2/3':2, 
                    'Danno D2-D3 Medio-Grave:>2/3':6,
                    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3':6,
                    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3':3,
                    'Danno D2-D3 Medio-Grave:1/3-2/3, Danno D1 Leggero:<1/3':5,
                    'Danno D4-D5 Gravissimo:>2/3':15, #rule exeption
                    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:1/3-2/3':12,
                    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:>2/3':10,
                    'Danno D1 Leggero:>2/3':3,
                    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:>2/3':5,
                    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3':7,
                    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:1/3-2/3':4,
                    'Danno D4-D5 Gravissimo:>2/3, Danno D2-D3 Medio-Grave:<1/3':14,
                    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:1/3-2/3':6,
                    'Danno D4-D5 Gravissimo:>2/3, Danno D1 Leggero:<1/3':13,
                    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D1 Leggero:1/3-2/3':9,
                    'nan':0}
damage_scores_dict = {
    'Danno Nullo': 0,
    'nan': 0,
    'Danno D1 Leggero:<1/3': 1,
    'Danno D1 Leggero:1/3-2/3': 2,
    'Danno D2-D3 Medio-Grave:<1/3': 3,
    'Danno D1 Leggero:>2/3': 3,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 4,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:1/3-2/3': 4,
    'Danno D2-D3 Medio-Grave:1/3-2/3': 6,
    'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:>2/3': 6,
    'Danno D2-D3 Medio-Grave:1/3-2/3, Danno D1 Leggero:<1/3': 6,
    'Danno D4-D5 Gravissimo:<1/3': 8,
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:<1/3': 9,
    'Danno D2-D3 Medio-Grave:>2/3': 9,
    'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:1/3-2/3': 10,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3': 10,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 10,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:1/3-2/3': 10,
    'Danno D4-D5 Gravissimo:1/3-2/3': 14,
    'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:>2/3': 9,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D1 Leggero:1/3-2/3': 14,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:<1/3': 14,
    'Danno D4-D5 Gravissimo:1/3-2/3, Danno D2-D3 Medio-Grave:1/3-2/3': 14,
    'Danno D4-D5 Gravissimo:>2/3, Danno D1 Leggero:<1/3': 19,
    'Danno D4-D5 Gravissimo:>2/3, Danno D2-D3 Medio-Grave:<1/3': 19,
    'Danno D4-D5 Gravissimo:>2/3': 19
    }
#sort dictionary by value
damage_scores_dict = {k: v for k, v in sorted(damage_scores_dict.items(), key=lambda item: item[1])}
damage_scores_dict

{'Danno Nullo': 0,
 'nan': 0,
 'Danno D1 Leggero:<1/3': 1,
 'Danno D1 Leggero:1/3-2/3': 2,
 'Danno D2-D3 Medio-Grave:<1/3': 3,
 'Danno D1 Leggero:>2/3': 3,
 'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 4,
 'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:1/3-2/3': 4,
 'Danno D2-D3 Medio-Grave:1/3-2/3': 6,
 'Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:>2/3': 6,
 'Danno D2-D3 Medio-Grave:1/3-2/3, Danno D1 Leggero:<1/3': 6,
 'Danno D4-D5 Gravissimo:<1/3': 8,
 'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:<1/3': 9,
 'Danno D2-D3 Medio-Grave:>2/3': 9,
 'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:>2/3': 9,
 'Danno D4-D5 Gravissimo:<1/3, Danno D1 Leggero:1/3-2/3': 10,
 'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3': 10,
 'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:<1/3, Danno D1 Leggero:<1/3': 10,
 'Danno D4-D5 Gravissimo:<1/3, Danno D2-D3 Medio-Grave:1/3-2/3': 10,
 'Danno D4-D5 Gravissimo:1/3-2/3': 14,
 'Danno D4-D5 Gravissimo:1/3-2/3, Danno D1 L